# 3D Skeleton visualization
Instructions:
1. Run cell 1 below
2. rotate the plot so that:
`
-y is up,
+x is right,
+z is forward.
`
3. Run cell 2 below to start tracking
4. The skeleton should now appear in the correct orientation in the output of cell 1 below.


In [19]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import plotly.graph_objs as go

# Visualization setup
lines = []
fig = go.FigureWidget()
for index, connection in enumerate(mp_pose.POSE_CONNECTIONS):
    # default value init, to make python happy with null refs
    lines.append([[0, 0], [1, 2], [-1, 1]])
    fig.add_scatter3d(x=lines[index][0], y=lines[index][1], z=lines[index][2], mode='lines')

fig

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter3d',
              'uid': 'f61ac558-4d2a-4717-9519-97d36d509964',
              'x': [0, 0],
              'y': [1, 2],
              'z': [-1, 1]},
             {'mode': 'lines',
              'type': 'scatter3d',
              'uid': '66199e54-7146-4112-b745-b811160f1ddb',
              'x': [0, 0],
              'y': [1, 2],
              'z': [-1, 1]},
             {'mode': 'lines',
              'type': 'scatter3d',
              'uid': 'f2017abb-442c-4ca6-89b3-ddb51c64dfa0',
              'x': [0, 0],
              'y': [1, 2],
              'z': [-1, 1]},
             {'mode': 'lines',
              'type': 'scatter3d',
              'uid': '25ba5481-a4cb-45dc-828f-140f6a65dc1b',
              'x': [0, 0],
              'y': [1, 2],
              'z': [-1, 1]},
             {'mode': 'lines',
              'type': 'scatter3d',
              'uid': '630763d8-51fe-4384-adfc-86811cd85958',
              'x':

In [22]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Visualization
        success = False
        try:
            landmarks = results.pose_world_landmarks.landmark
            success = True
        except:
            success = False

        if success:
            xs = []
            ys = []
            zs = []
            # fig = go.FigureWidget()
            with fig.batch_update():
                for index, connection in enumerate(mp_pose.POSE_CONNECTIONS):
                    # Each connection:
                    joint_a = landmarks[connection[0]]
                    joint_b = landmarks[connection[1]]
                    # fig.add_scatter3d(x=[joint_a.x, joint_b.x], y=[joint_a.y, joint_b.y], z=[joint_a.z, joint_b.z], mode='lines')
                    # xs.append([joint_a.x, joint_b.x])
                    # ys.append([joint_a.y, joint_b.y])
                    # zs.append([joint_a.z, joint_b.z])
                    fig.data[index].x = [joint_a.x, joint_b.x]
                    fig.data[index].y = [joint_a.y, joint_b.y]
                    fig.data[index].z = [joint_a.z, joint_b.z]
                    # print(fig.data[index])
            # fig.show()
            # break
            # fig.show()

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()